# 🥈 Silver Layer - Data Cleansing & Transformation

**NYC Taxi Data Engineering Project - Microsoft Fabric**

This notebook transforms raw Bronze data into clean, validated Silver tables with:
- Data type standardization
- Quality filtering
- Enrichment with derived columns
- Delta Lake format with optimization

## ⚠️ Schema Handling
Reads each parquet file individually to handle schema differences between months.

## 1. Setup & Configuration

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from delta.tables import DeltaTable
import pandas as pd

# Configuration
BRONZE_PATH = "Files/bronze"
SILVER_PATH = "Tables"

# Date range filter
START_DATE = "2021-01-01"
END_DATE = "2025-12-31"

print("✅ Setup completed")
print(f"📅 Processing date range: {START_DATE} to {END_DATE}")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 3, Finished, Available, Finished)

✅ Setup completed
📅 Processing date range: 2021-01-01 to 2025-12-31


## 2. Transform Taxi Zones (Reference Data)

In [2]:
# Read raw zones data
print("🗺️ Loading Taxi Zones...")

try:
    df_zones_raw = spark.read.option("header", "true").option("inferSchema", "true") \
        .csv(f"{BRONZE_PATH}/taxi_zones/taxi_zone_lookup.csv")
except:
    try:
        df_zones_raw = spark.read.option("header", "true").option("inferSchema", "true") \
            .csv(f"{BRONZE_PATH}/taxi_zones/*.csv")
    except:
        df_zones_raw = spark.read.option("header", "true").option("inferSchema", "true") \
            .csv(f"{BRONZE_PATH}/taxi_zones")

print(f"📋 Columns: {df_zones_raw.columns}")

# Transform and clean
df_zones_silver = df_zones_raw.select(
    F.col("LocationID").cast("int").alias("location_id"),
    F.trim(F.col("Borough")).alias("borough"),
    F.trim(F.col("Zone")).alias("zone_name"),
    F.trim(F.col("service_zone")).alias("service_zone")
).filter(
    F.col("location_id").isNotNull()
).withColumn(
    "borough_clean",
    F.when(F.col("borough") == "EWR", "Newark Airport")
    .when(F.col("borough") == "Unknown", "Unknown")
    .otherwise(F.col("borough"))
).withColumn(
    "is_airport",
    F.when(F.col("zone_name").contains("Airport"), True)
    .when(F.col("borough") == "EWR", True)
    .otherwise(False)
).withColumn(
    "zone_type",
    F.when(F.col("is_airport"), "Airport")
    .when(F.col("borough_clean") == "Manhattan", "Manhattan")
    .when(F.col("borough_clean").isin("Brooklyn", "Queens", "Bronx", "Staten Island"), "Outer Borough")
    .otherwise("Other")
).drop("borough").withColumnRenamed("borough_clean", "borough")

print(f"📊 Taxi Zones Silver: {df_zones_silver.count()} records")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 4, Finished, Available, Finished)

🗺️ Loading Taxi Zones...
📋 Columns: ['LocationID', 'Borough', 'Zone', 'service_zone']
📊 Taxi Zones Silver: 265 records


In [3]:
# Save to Silver layer
df_zones_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("silver_taxi_zones")

print("✅ silver_taxi_zones table created")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 5, Finished, Available, Finished)

✅ silver_taxi_zones table created


In [7]:
df = spark.sql("SELECT * FROM TaxiAnalytics_Lakehouse.dbo.silver_taxi_zones LIMIT 1000")
display(df)

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6738695e-ce57-4db3-b8a0-74dca69077db)

## 3. Transform Weather Data

In [4]:
# Read raw weather data
print("🌤️ Loading Weather Data...")

try:
    df_weather_raw = spark.read.option("header", "true").option("inferSchema", "true") \
        .csv(f"{BRONZE_PATH}/weather/central_park_weather.csv")
except:
    try:
        df_weather_raw = spark.read.option("header", "true").option("inferSchema", "true") \
            .csv(f"{BRONZE_PATH}/weather/*.csv")
    except:
        df_weather_raw = spark.read.option("header", "true").option("inferSchema", "true") \
            .csv(f"{BRONZE_PATH}/weather")

print(f"📋 Available columns: {df_weather_raw.columns}")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 6, Finished, Available, Finished)

🌤️ Loading Weather Data...
📋 Available columns: ['station', 'name', 'date', 'awnd', 'prcp', 'snow', 'snwd', 'tmax', 'tmin']


In [5]:
# Check if tavg exists, if not calculate from tmax and tmin
weather_cols = [c.lower() for c in df_weather_raw.columns]

if "tavg" in weather_cols:
    # tavg exists
    temp_avg_expr = F.col("tavg").cast("double")
else:
    # Calculate from tmax and tmin
    print("⚠️ Column 'tavg' not found, calculating from tmax and tmin")
    temp_avg_expr = ((F.col("tmax").cast("double") + F.col("tmin").cast("double")) / 2)

# Transform weather data
df_weather_silver = df_weather_raw.select(
    F.to_date(F.col("date")).alias("date"),
    temp_avg_expr.alias("temp_avg"),
    F.col("tmax").cast("double").alias("temp_max"),
    F.col("tmin").cast("double").alias("temp_min"),
    F.coalesce(F.col("prcp"), F.lit(0.0)).cast("double").alias("precipitation"),
    F.coalesce(F.col("snow"), F.lit(0.0)).cast("double").alias("snowfall"),
    F.coalesce(F.col("snwd"), F.lit(0.0)).cast("double").alias("snow_depth"),
    F.coalesce(F.col("awnd"), F.lit(0.0)).cast("double").alias("wind_speed")
).filter(
    F.col("date").isNotNull()
).withColumn(
    "weather_condition",
    F.when(F.col("snowfall") > 0, "Snow")
    .when(F.col("precipitation") > 0.5, "Heavy Rain")
    .when(F.col("precipitation") > 0, "Rain")
    .when(F.col("wind_speed") > 25, "Windy")
    .otherwise("Clear")
).withColumn(
    "temp_range",
    F.when(F.col("temp_avg") < 32, "Freezing")
    .when(F.col("temp_avg") < 50, "Cold")
    .when(F.col("temp_avg") < 68, "Mild")
    .when(F.col("temp_avg") < 86, "Warm")
    .otherwise("Hot")
).withColumn(
    "is_severe_weather",
    F.when(
        (F.col("snowfall") > 3) | 
        (F.col("precipitation") > 1) | 
        (F.col("wind_speed") > 35), 
        True
    ).otherwise(False)
)

print(f"📊 Weather Silver: {df_weather_silver.count()} records")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 7, Finished, Available, Finished)

⚠️ Column 'tavg' not found, calculating from tmax and tmin
📊 Weather Silver: 1155 records


In [6]:
# Save weather to Silver
df_weather_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("silver_weather")

print("✅ silver_weather table created")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 8, Finished, Available, Finished)

✅ silver_weather table created


In [8]:
df = spark.sql("SELECT * FROM TaxiAnalytics_Lakehouse.dbo.silver_weather LIMIT 1000")
display(df)

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8290bdf5-89aa-4f6b-b954-327896883240)

## 4. Transform Holidays Data

In [10]:
# Read raw holidays data
print("🎄 Loading Holidays Data...")

try:
    df_holidays_raw = spark.read.option("header", "true").option("inferSchema", "true") \
        .csv(f"{BRONZE_PATH}/holidays/us_holidays.csv")
except:
    try:
        df_holidays_raw = spark.read.option("header", "true").option("inferSchema", "true") \
            .csv(f"{BRONZE_PATH}/holidays/*.csv")
    except:
        df_holidays_raw = spark.read.option("header", "true").option("inferSchema", "true") \
            .csv(f"{BRONZE_PATH}/holidays")

print(f"📋 Columns: {df_holidays_raw.columns}")

# Dynamic column mapping based on actual columns
cols_lower = {c.lower(): c for c in df_holidays_raw.columns}

# Find the right column names
date_col = cols_lower.get("date", "date")
holiday_col = cols_lower.get("holiday_name", cols_lower.get("holiday", "holiday_name"))
type_col = cols_lower.get("type", cols_lower.get("holiday_type", None))

# Transform holidays
df_holidays_silver = df_holidays_raw.select(
    F.to_date(F.col(date_col)).alias("date"),
    F.trim(F.col(holiday_col)).alias("holiday_name")
).filter(
    F.col("date").isNotNull()
)

# Add holiday_type if exists, otherwise set default
if type_col and type_col in df_holidays_raw.columns:
    df_holidays_silver = df_holidays_silver.withColumn(
        "holiday_type", 
        F.trim(F.col(type_col))
    )
else:
    df_holidays_silver = df_holidays_silver.withColumn(
        "holiday_type", 
        F.lit("Federal")
    )

# Add derived columns
df_holidays_silver = df_holidays_silver.withColumn(
    "is_major_holiday",
    F.when(
        F.col("holiday_name").isin(
            "New Year's Day", "Independence Day", 
            "Thanksgiving", "Christmas Day"
        ), True
    ).otherwise(False)
).withColumn(
    "holiday_category",
    F.when(F.col("holiday_name").contains("Christmas"), "Winter")
    .when(F.col("holiday_name").contains("Thanksgiving"), "Fall")
    .when(F.col("holiday_name").contains("Independence"), "Summer")
    .when(F.col("holiday_name").contains("Memorial"), "Spring")
    .when(F.col("holiday_name").contains("Labor"), "Fall")
    .otherwise("Other")
)

print(f"📊 Holidays Silver: {df_holidays_silver.count()} records")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 12, Finished, Available, Finished)

🎄 Loading Holidays Data...
📋 Columns: ['date', 'holiday_name', 'year']
📊 Holidays Silver: 75 records


In [11]:
# Save holidays to Silver
df_holidays_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("silver_holidays")

print("✅ silver_holidays table created")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 13, Finished, Available, Finished)

✅ silver_holidays table created


In [12]:
df = spark.sql("SELECT * FROM TaxiAnalytics_Lakehouse.dbo.silver_holidays LIMIT 1000")
display(df)

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a8337f29-d82d-405e-ba53-9f3271838cec)

## 5. Create Calendar Dimension

In [13]:
# Generate date range
print("📅 Creating Calendar Dimension...")

df_calendar = spark.sql(f"""
    SELECT explode(sequence(
        to_date('{START_DATE}'), 
        to_date('{END_DATE}'), 
        interval 1 day
    )) as date
""")

# Add calendar attributes
df_calendar_silver = df_calendar.select(
    F.col("date"),
    F.year("date").alias("year"),
    F.quarter("date").alias("quarter"),
    F.month("date").alias("month"),
    F.dayofmonth("date").alias("day"),
    F.dayofweek("date").alias("day_of_week"),
    F.weekofyear("date").alias("week_of_year"),
    F.date_format("date", "EEEE").alias("day_name"),
    F.date_format("date", "MMMM").alias("month_name"),
    F.when(F.dayofweek("date").isin(1, 7), True).otherwise(False).alias("is_weekend")
)

# Join with holidays
df_calendar_silver = df_calendar_silver.join(
    df_holidays_silver.select("date", "holiday_name", "is_major_holiday"),
    on="date",
    how="left"
).withColumn(
    "is_holiday",
    F.when(F.col("holiday_name").isNotNull(), True).otherwise(False)
)

print(f"📊 Calendar Silver: {df_calendar_silver.count()} records")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 15, Finished, Available, Finished)

📅 Creating Calendar Dimension...
📊 Calendar Silver: 1826 records


In [14]:
# Save calendar to Silver
df_calendar_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("silver_calendar")

print("✅ silver_calendar table created")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 16, Finished, Available, Finished)

✅ silver_calendar table created


In [15]:
df = spark.sql("SELECT * FROM TaxiAnalytics_Lakehouse.dbo.silver_calendar LIMIT 1000")
display(df)

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e7bf5464-d8b8-4c43-9c82-0f8badcc45eb)

## 6. Transform NYC Taxi Trips (Main Fact Data)

### ⚠️ Schema Handling
Read each parquet file individually to handle schema differences.

### 6.1 List and Load All Parquet Files

In [16]:
# Get list of all parquet files
TAXI_PATH = f"{BRONZE_PATH}/taxi_trips"

print("📂 Scanning for parquet files...")
df_file_list = spark.read.format("binaryFile").option("pathGlobFilter", "*.parquet").load(TAXI_PATH)
file_paths = [row.path for row in df_file_list.select("path").collect()]

print(f"📊 Found {len(file_paths)} parquet files")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 18, Finished, Available, Finished)

📂 Scanning for parquet files...
📊 Found 59 parquet files


In [17]:
def read_and_normalize_single_file(file_path):
    """
    Read a single parquet file and normalize its schema.
    """
    file_name = file_path.split('/')[-1]
    
    try:
        df_raw = spark.read.parquet(file_path)
        
        # Find airport_fee column (could be Airport_fee)
        airport_fee_col = None
        for c in df_raw.columns:
            if c.lower() == "airport_fee":
                airport_fee_col = c
                break
        
        # Normalize schema using selectExpr
        df_normalized = df_raw.selectExpr(
            "CAST(VendorID AS LONG) AS vendor_id",
            "CAST(tpep_pickup_datetime AS TIMESTAMP) AS pickup_datetime",
            "CAST(tpep_dropoff_datetime AS TIMESTAMP) AS dropoff_datetime",
            "CAST(passenger_count AS DOUBLE) AS passenger_count",
            "CAST(trip_distance AS DOUBLE) AS trip_distance",
            "CAST(RatecodeID AS DOUBLE) AS rate_code_id",
            "CAST(store_and_fwd_flag AS STRING) AS store_and_fwd_flag",
            "CAST(PULocationID AS INT) AS pickup_location_id",
            "CAST(DOLocationID AS INT) AS dropoff_location_id",
            "CAST(payment_type AS INT) AS payment_type",
            "CAST(fare_amount AS DOUBLE) AS fare_amount",
            "CAST(extra AS DOUBLE) AS extra",
            "CAST(mta_tax AS DOUBLE) AS mta_tax",
            "CAST(tip_amount AS DOUBLE) AS tip_amount",
            "CAST(tolls_amount AS DOUBLE) AS tolls_amount",
            "CAST(improvement_surcharge AS DOUBLE) AS improvement_surcharge",
            "CAST(total_amount AS DOUBLE) AS total_amount",
            "CAST(COALESCE(congestion_surcharge, 0.0) AS DOUBLE) AS congestion_surcharge",
            f"CAST(COALESCE({airport_fee_col if airport_fee_col else 'NULL'}, 0.0) AS DOUBLE) AS airport_fee"
        )
        
        return df_normalized
        
    except Exception as e:
        print(f"    ❌ Error: {file_name}: {str(e)[:80]}")
        return None

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 19, Finished, Available, Finished)

In [18]:
# Load all files one by one
print("🔄 Loading taxi files one by one...")
print("=" * 70)

all_dataframes = []

for i, file_path in enumerate(sorted(file_paths)):
    file_name = file_path.split('/')[-1]
    
    df_normalized = read_and_normalize_single_file(file_path)
    
    if df_normalized is not None:
        all_dataframes.append(df_normalized)
        print(f"  ✅ [{i+1:2}/{len(file_paths)}] {file_name}")
    else:
        print(f"  ❌ [{i+1:2}/{len(file_paths)}] {file_name}: FAILED")

print("=" * 70)
print(f"📊 Successfully loaded {len(all_dataframes)} files")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 20, Finished, Available, Finished)

🔄 Loading taxi files one by one...
  ✅ [ 1/59] yellow_tripdata_2021-01.parquet
  ✅ [ 2/59] yellow_tripdata_2021-02.parquet
  ✅ [ 3/59] yellow_tripdata_2021-03.parquet
  ✅ [ 4/59] yellow_tripdata_2021-04.parquet
  ✅ [ 5/59] yellow_tripdata_2021-05.parquet
  ✅ [ 6/59] yellow_tripdata_2021-06.parquet
  ✅ [ 7/59] yellow_tripdata_2021-07.parquet
  ✅ [ 8/59] yellow_tripdata_2021-08.parquet
  ✅ [ 9/59] yellow_tripdata_2021-09.parquet
  ✅ [10/59] yellow_tripdata_2021-10.parquet
  ✅ [11/59] yellow_tripdata_2021-11.parquet
  ✅ [12/59] yellow_tripdata_2021-12.parquet
  ✅ [13/59] yellow_tripdata_2022-01.parquet
  ✅ [14/59] yellow_tripdata_2022-02.parquet
  ✅ [15/59] yellow_tripdata_2022-03.parquet
  ✅ [16/59] yellow_tripdata_2022-04.parquet
  ✅ [17/59] yellow_tripdata_2022-05.parquet
  ✅ [18/59] yellow_tripdata_2022-06.parquet
  ✅ [19/59] yellow_tripdata_2022-07.parquet
  ✅ [20/59] yellow_tripdata_2022-08.parquet
  ✅ [21/59] yellow_tripdata_2022-09.parquet
  ✅ [22/59] yellow_tripdata_2022-10.parqu

In [19]:
# Union all dataframes
print("🔄 Merging all DataFrames...")

df_taxi_raw = all_dataframes[0]
for df in all_dataframes[1:]:
    df_taxi_raw = df_taxi_raw.unionAll(df)

print(f"✅ Merged. Schema:")
df_taxi_raw.printSchema()

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 21, Finished, Available, Finished)

🔄 Merging all DataFrames...
✅ Merged. Schema:
root
 |-- vendor_id: long (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- rate_code_id: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = false)
 |-- airport_fee: double (nullable = false)



### 6.2 Apply Data Quality Rules

In [20]:
print("🧹 Applying data quality rules...")

df_taxi_clean = df_taxi_raw.filter(
    # Date range filter
    (F.col("pickup_datetime") >= START_DATE) & 
    (F.col("pickup_datetime") <= END_DATE) &
    
    # Valid pickup and dropoff times
    (F.col("pickup_datetime").isNotNull()) &
    (F.col("dropoff_datetime").isNotNull()) &
    (F.col("dropoff_datetime") > F.col("pickup_datetime")) &
    
    # Valid location IDs (1-265)
    (F.col("pickup_location_id").between(1, 265)) &
    (F.col("dropoff_location_id").between(1, 265)) &
    
    # Valid trip distance (0 to 500 miles)
    (F.col("trip_distance") >= 0) &
    (F.col("trip_distance") <= 500) &
    
    # Valid fare amount ($0 to $5000)
    (F.col("fare_amount") >= 0) &
    (F.col("fare_amount") <= 5000) &
    
    # Valid total amount
    (F.col("total_amount") >= 0) &
    (F.col("total_amount") <= 10000) &
    
    # Trip duration < 24 hours and > 0
    ((F.unix_timestamp("dropoff_datetime") - F.unix_timestamp("pickup_datetime")) < 86400) &
    ((F.unix_timestamp("dropoff_datetime") - F.unix_timestamp("pickup_datetime")) > 0)
)

# Cache for faster operations
df_taxi_clean.cache()
clean_count = df_taxi_clean.count()
print(f"📊 Records after quality filtering: {clean_count:,}")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 22, Finished, Available, Finished)

🧹 Applying data quality rules...
📊 Records after quality filtering: 189,523,210


### 6.3 Add Derived Columns

In [21]:
print("✨ Adding derived columns...")

df_taxi_enriched = df_taxi_clean \
    .withColumn("pickup_date", F.to_date("pickup_datetime")) \
    .withColumn("pickup_year", F.year("pickup_datetime")) \
    .withColumn("pickup_month", F.month("pickup_datetime")) \
    .withColumn("pickup_day", F.dayofmonth("pickup_datetime")) \
    .withColumn("pickup_hour", F.hour("pickup_datetime")) \
    .withColumn("pickup_dayofweek", F.dayofweek("pickup_datetime")) \
    .withColumn(
        "trip_duration_minutes",
        F.round((F.unix_timestamp("dropoff_datetime") - F.unix_timestamp("pickup_datetime")) / 60, 2)
    ) \
    .withColumn(
        "avg_speed_mph",
        F.when(
            F.col("trip_duration_minutes") > 0,
            F.round(F.col("trip_distance") / (F.col("trip_duration_minutes") / 60), 2)
        ).otherwise(0)
    ) \
    .withColumn(
        "time_of_day",
        F.when((F.hour("pickup_datetime") >= 5) & (F.hour("pickup_datetime") < 12), "Morning")
        .when((F.hour("pickup_datetime") >= 12) & (F.hour("pickup_datetime") < 17), "Afternoon")
        .when((F.hour("pickup_datetime") >= 17) & (F.hour("pickup_datetime") < 21), "Evening")
        .otherwise("Night")
    ) \
    .withColumn(
        "is_weekend",
        F.when(F.dayofweek("pickup_datetime").isin(1, 7), True).otherwise(False)
    ) \
    .withColumn(
        "is_rush_hour",
        F.when(
            (~F.col("is_weekend")) & 
            (((F.hour("pickup_datetime") >= 7) & (F.hour("pickup_datetime") <= 9)) |
             ((F.hour("pickup_datetime") >= 17) & (F.hour("pickup_datetime") <= 19))),
            True
        ).otherwise(False)
    ) \
    .withColumn(
        "distance_category",
        F.when(F.col("trip_distance") <= 1, "Short")
        .when(F.col("trip_distance") <= 5, "Medium")
        .when(F.col("trip_distance") <= 15, "Long")
        .otherwise("Very Long")
    ) \
    .withColumn(
        "tip_percentage",
        F.when(
            (F.col("payment_type") == 1) & (F.col("fare_amount") > 0),
            F.round((F.col("tip_amount") / F.col("fare_amount")) * 100, 2)
        ).otherwise(0)
    ) \
    .withColumn(
        "vendor_name",
        F.when(F.col("vendor_id") == 1, "CMT")
        .when(F.col("vendor_id") == 2, "VeriFone")
        .otherwise("Unknown")
    ) \
    .withColumn(
        "payment_type_name",
        F.when(F.col("payment_type") == 1, "Credit Card")
        .when(F.col("payment_type") == 2, "Cash")
        .when(F.col("payment_type") == 3, "No Charge")
        .when(F.col("payment_type") == 4, "Dispute")
        .when(F.col("payment_type") == 5, "Unknown")
        .when(F.col("payment_type") == 6, "Voided")
        .otherwise("Other")
    ) \
    .withColumn(
        "rate_code_name",
        F.when(F.col("rate_code_id") == 1, "Standard")
        .when(F.col("rate_code_id") == 2, "JFK")
        .when(F.col("rate_code_id") == 3, "Newark")
        .when(F.col("rate_code_id") == 4, "Nassau/Westchester")
        .when(F.col("rate_code_id") == 5, "Negotiated")
        .when(F.col("rate_code_id") == 6, "Group Ride")
        .otherwise("Unknown")
    )

print(f"📊 Enriched Records: {df_taxi_enriched.count():,}")
print(f"📋 Total Columns: {len(df_taxi_enriched.columns)}")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 23, Finished, Available, Finished)

✨ Adding derived columns...
📊 Enriched Records: 189,523,210
📋 Total Columns: 35


### 6.4 Save to Silver Layer

In [22]:
print("💾 Saving to silver_taxi_trips (partitioned by year/month)...")

df_taxi_enriched.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .partitionBy("pickup_year", "pickup_month") \
    .saveAsTable("silver_taxi_trips")

print("✅ silver_taxi_trips table created")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 24, Finished, Available, Finished)

💾 Saving to silver_taxi_trips (partitioned by year/month)...
✅ silver_taxi_trips table created


In [23]:
df = spark.sql("SELECT * FROM TaxiAnalytics_Lakehouse.dbo.silver_taxi_trips LIMIT 1000")
display(df)

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 25, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 280bd224-f362-49a7-bde5-48905457a1f0)

### 6.5 Optimize Table

In [24]:
print("⚡ Optimizing silver_taxi_trips with Z-ORDER...")

spark.sql("OPTIMIZE silver_taxi_trips ZORDER BY (pickup_date, pickup_location_id)")

print("✅ Optimization complete")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 26, Finished, Available, Finished)

⚡ Optimizing silver_taxi_trips with Z-ORDER...
✅ Optimization complete


In [25]:
df = spark.sql("SELECT * FROM TaxiAnalytics_Lakehouse.dbo.silver_taxi_trips LIMIT 1000")
display(df)

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 27, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6d39e161-a657-47e1-a71c-77b892a3456a)

## 7. Validation & Summary

In [26]:
# Validate all Silver tables
print("=" * 80)
print("📋 SILVER LAYER SUMMARY")
print("=" * 80)

tables = [
    "silver_taxi_trips", 
    "silver_taxi_zones", 
    "silver_weather", 
    "silver_holidays", 
    "silver_calendar"
]

for table in tables:
    try:
        count = spark.table(table).count()
        cols = len(spark.table(table).columns)
        print(f"  ✅ {table:25} │ {count:>15,} records │ {cols:>3} columns")
    except Exception as e:
        print(f"  ❌ {table:25} │ Error: {e}")

print("=" * 80)

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 28, Finished, Available, Finished)

📋 SILVER LAYER SUMMARY
  ✅ silver_taxi_trips         │     189,523,210 records │  35 columns
  ✅ silver_taxi_zones         │             265 records │   6 columns
  ✅ silver_weather            │           1,155 records │  11 columns
  ✅ silver_holidays           │              75 records │   5 columns
  ✅ silver_calendar           │           1,826 records │  13 columns


In [27]:
# Sample data from silver_taxi_trips
print("\n📋 Sample Silver Taxi Trips:")

spark.table("silver_taxi_trips").select(
    "pickup_datetime", 
    "pickup_location_id", 
    "dropoff_location_id",
    "trip_distance", 
    "fare_amount", 
    "tip_amount", 
    "total_amount",
    "trip_duration_minutes", 
    "time_of_day", 
    "is_weekend",
    "distance_category"
).show(5, truncate=False)

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 29, Finished, Available, Finished)


📋 Sample Silver Taxi Trips:
+-------------------+------------------+-------------------+-------------+-----------+----------+------------+---------------------+-----------+----------+-----------------+
|pickup_datetime    |pickup_location_id|dropoff_location_id|trip_distance|fare_amount|tip_amount|total_amount|trip_duration_minutes|time_of_day|is_weekend|distance_category|
+-------------------+------------------+-------------------+-------------+-----------+----------+------------+---------------------+-----------+----------+-----------------+
|2022-08-01 00:17:39|114               |148                |0.4          |3.5        |1.8       |9.1         |2.32                 |Night      |false     |Short            |
|2022-08-01 00:26:06|79                |137                |1.4          |6.5        |2.05      |12.35       |5.82                 |Night      |false     |Medium           |
|2022-08-01 00:45:49|79                |74                 |5.2          |16.0       |3.95      |23.7

In [28]:
# Records by year
print("📅 Silver Taxi Trips by Year:")

spark.table("silver_taxi_trips") \
    .groupBy("pickup_year") \
    .agg(
        F.count("*").alias("trips"),
        F.round(F.sum("total_amount"), 2).alias("revenue")
    ) \
    .orderBy("pickup_year") \
    .show()

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 30, Finished, Available, Finished)

📅 Silver Taxi Trips by Year:
+-----------+--------+---------------+
|pickup_year|   trips|        revenue|
+-----------+--------+---------------+
|       2021|30702566| 6.0742144291E8|
|       2022|39366536| 8.6161260907E8|
|       2023|37910042|1.09884095487E9|
|       2024|40421123|1.16136209065E9|
|       2025|41122943|1.17876133914E9|
+-----------+--------+---------------+



## 8. Bronze vs Silver Comparison

In [31]:
# ============================================================================
# 1. TAXI TRIPS COMPARISON
# ============================================================================
print("\n" + "─" * 90)
print("🚕 TAXI TRIPS")
print("─" * 90)

# Bronze: Count from raw files (already loaded in all_dataframes)
bronze_taxi_count = sum([df.count() for df in all_dataframes])

# Silver: Count from table
silver_taxi_count = spark.table("silver_taxi_trips").count()

# Calculate difference
taxi_removed = bronze_taxi_count - silver_taxi_count
taxi_removed_pct = (taxi_removed / bronze_taxi_count) * 100 if bronze_taxi_count > 0 else 0

print(f"  {'Bronze (Raw):':<25} {bronze_taxi_count:>20,} records")
print(f"  {'Silver (Cleaned):':<25} {silver_taxi_count:>20,} records")
print(f"  {'Removed by Quality Rules:':<25} {taxi_removed:>20,} records ({taxi_removed_pct:.2f}%)")
print(f"  {'Columns Added:':<25} {'15 derived columns':>20}")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 33, Finished, Available, Finished)


──────────────────────────────────────────────────────────────────────────────────────────
🚕 TAXI TRIPS
──────────────────────────────────────────────────────────────────────────────────────────
  Bronze (Raw):                      194,457,948 records
  Silver (Cleaned):                  189,523,210 records
  Removed by Quality Rules:            4,934,738 records (2.54%)
  Columns Added:              15 derived columns


In [32]:
# ============================================================================
# 2. TAXI ZONES COMPARISON
# ============================================================================
print("\n" + "─" * 90)
print("🗺️ TAXI ZONES")
print("─" * 90)

bronze_zones_count = df_zones_raw.count()
silver_zones_count = spark.table("silver_taxi_zones").count()
bronze_zones_cols = len(df_zones_raw.columns)
silver_zones_cols = len(spark.table("silver_taxi_zones").columns)

print(f"  {'Bronze Records:':<25} {bronze_zones_count:>20,}")
print(f"  {'Silver Records:':<25} {silver_zones_count:>20,}")
print(f"  {'Bronze Columns:':<25} {bronze_zones_cols:>20}")
print(f"  {'Silver Columns:':<25} {silver_zones_cols:>20}")
print(f"  {'New Columns:':<25} {'is_airport, zone_type':>20}")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 34, Finished, Available, Finished)


──────────────────────────────────────────────────────────────────────────────────────────
🗺️ TAXI ZONES
──────────────────────────────────────────────────────────────────────────────────────────
  Bronze Records:                            265
  Silver Records:                            265
  Bronze Columns:                              4
  Silver Columns:                              6
  New Columns:              is_airport, zone_type


In [33]:
# ============================================================================
# 3. WEATHER COMPARISON
# ============================================================================
print("\n" + "─" * 90)
print("🌤️ WEATHER DATA")
print("─" * 90)

bronze_weather_count = df_weather_raw.count()
silver_weather_count = spark.table("silver_weather").count()
bronze_weather_cols = len(df_weather_raw.columns)
silver_weather_cols = len(spark.table("silver_weather").columns)

print(f"  {'Bronze Records:':<25} {bronze_weather_count:>20,}")
print(f"  {'Silver Records:':<25} {silver_weather_count:>20,}")
print(f"  {'Bronze Columns:':<25} {bronze_weather_cols:>20}")
print(f"  {'Silver Columns:':<25} {silver_weather_cols:>20}")
print(f"  {'New Columns:':<25} {'weather_condition, temp_range, is_severe':>20}")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 35, Finished, Available, Finished)


──────────────────────────────────────────────────────────────────────────────────────────
🌤️ WEATHER DATA
──────────────────────────────────────────────────────────────────────────────────────────
  Bronze Records:                          1,155
  Silver Records:                          1,155
  Bronze Columns:                              9
  Silver Columns:                             11
  New Columns:              weather_condition, temp_range, is_severe


In [35]:
# ============================================================================
# 4. HOLIDAYS COMPARISON
# ============================================================================
print("\n" + "─" * 90)
print("🎄 HOLIDAYS DATA")
print("─" * 90)

bronze_holidays_count = df_holidays_raw.count()
silver_holidays_count = spark.table("silver_holidays").count()
bronze_holidays_cols = len(df_holidays_raw.columns)
silver_holidays_cols = len(spark.table("silver_holidays").columns)

print(f"  {'Bronze Records:':<25} {bronze_holidays_count:>20,}")
print(f"  {'Silver Records:':<25} {silver_holidays_count:>20,}")
print(f"  {'Bronze Columns:':<25} {bronze_holidays_cols:>20}")
print(f"  {'Silver Columns:':<25} {silver_holidays_cols:>20}")
print(f"  {'New Columns:':<25} {'is_major_holiday, holiday_category':>20}")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 37, Finished, Available, Finished)


──────────────────────────────────────────────────────────────────────────────────────────
🎄 HOLIDAYS DATA
──────────────────────────────────────────────────────────────────────────────────────────
  Bronze Records:                             75
  Silver Records:                             75
  Bronze Columns:                              3
  Silver Columns:                              5
  New Columns:              is_major_holiday, holiday_category


In [34]:
# ============================================================================
# 5. CALENDAR (NEW TABLE)
# ============================================================================
print("\n" + "─" * 90)
print("📅 CALENDAR DIMENSION (New in Silver)")
print("─" * 90)

silver_calendar_count = spark.table("silver_calendar").count()
silver_calendar_cols = len(spark.table("silver_calendar").columns)

print(f"  {'Bronze Records:':<25} {'N/A (generated)':>20}")
print(f"  {'Silver Records:':<25} {silver_calendar_count:>20,}")
print(f"  {'Silver Columns:':<25} {silver_calendar_cols:>20}")
print(f"  {'Date Range:':<25} {START_DATE + ' to ' + END_DATE:>20}")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 36, Finished, Available, Finished)


──────────────────────────────────────────────────────────────────────────────────────────
📅 CALENDAR DIMENSION (New in Silver)
──────────────────────────────────────────────────────────────────────────────────────────
  Bronze Records:                N/A (generated)
  Silver Records:                          1,826
  Silver Columns:                             13
  Date Range:               2021-01-01 to 2025-12-31


In [37]:
# ============================================================================
# SUMMARY TABLE
# ============================================================================
print("\n" + "=" * 90)
print("📋 SUMMARY TABLE")
print("=" * 90)

print(f"\n  {'Dataset':<15} │ {'Bronze Rows':>15} │ {'Silver Rows':>15} │ {'Bronze Cols':>12} │ {'Silver Cols':>12}")
print("  " + "─" * 75)
print(f"  {'Taxi Trips':<15} │ {bronze_taxi_count:>15,} │ {silver_taxi_count:>15,} │ {19:>12} │ {len(spark.table('silver_taxi_trips').columns):>12}")
print(f"  {'Taxi Zones':<15} │ {bronze_zones_count:>15,} │ {silver_zones_count:>15,} │ {bronze_zones_cols:>12} │ {silver_zones_cols:>12}")
print(f"  {'Weather':<15} │ {bronze_weather_count:>15,} │ {silver_weather_count:>15,} │ {bronze_weather_cols:>12} │ {silver_weather_cols:>12}")
print(f"  {'Holidays':<15} │ {bronze_holidays_count:>15,} │ {silver_holidays_count:>15,} │ {bronze_holidays_cols:>12} │ {silver_holidays_cols:>12}")
print(f"  {'Calendar':<15} │ {'N/A':>15} │ {silver_calendar_count:>15,} │ {'N/A':>12} │ {silver_calendar_cols:>12}")
print("  " + "─" * 75)

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 39, Finished, Available, Finished)


📋 SUMMARY TABLE

  Dataset         │     Bronze Rows │     Silver Rows │  Bronze Cols │  Silver Cols
  ───────────────────────────────────────────────────────────────────────────
  Taxi Trips      │     194,457,948 │     189,523,210 │           19 │           35
  Taxi Zones      │             265 │             265 │            4 │            6
  Weather         │           1,155 │           1,155 │            9 │           11
  Holidays        │              75 │              75 │            3 │            5
  Calendar        │             N/A │           1,826 │          N/A │           13
  ───────────────────────────────────────────────────────────────────────────


In [39]:
# ============================================================================
# DATA QUALITY IMPROVEMENTS
# ============================================================================
print("\n" + "=" * 90)
print("✨ DATA QUALITY IMPROVEMENTS IN SILVER LAYER")
print("=" * 90)

print(f"""
  🚕 Taxi Trips:
     • Removed {taxi_removed:,} invalid records ({taxi_removed_pct:.2f}%)
     • Filtered: invalid dates, negative values, outliers
     • Added: trip_duration, avg_speed, time_of_day, is_weekend, is_rush_hour, etc.

  🗺️ Taxi Zones:
     • Standardized borough names
     • Added: is_airport, zone_type

  🌤️ Weather:
     • Calculated temp_avg from tmax/tmin
     • Added: weather_condition, temp_range, is_severe_weather

  🎄 Holidays:
     • Added: is_major_holiday, holiday_category

  📅 Calendar:
     • Generated {silver_calendar_count:,} date records
     • Added: year, month, day, is_weekend, is_holiday
""")

print("=" * 90)
print("✅ Comparison Complete!")
print("=" * 90)

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 41, Finished, Available, Finished)


✨ DATA QUALITY IMPROVEMENTS IN SILVER LAYER

  🚕 Taxi Trips:
     • Removed 4,934,738 invalid records (2.54%)
     • Filtered: invalid dates, negative values, outliers
     • Added: trip_duration, avg_speed, time_of_day, is_weekend, is_rush_hour, etc.

  🗺️ Taxi Zones:
     • Standardized borough names
     • Added: is_airport, zone_type

  🌤️ Weather:
     • Calculated temp_avg from tmax/tmin
     • Added: weather_condition, temp_range, is_severe_weather

  🎄 Holidays:
     • Added: is_major_holiday, holiday_category

  📅 Calendar:
     • Generated 1,826 date records
     • Added: year, month, day, is_weekend, is_holiday

✅ Comparison Complete!


In [29]:
print("\n" + "=" * 80)
print("✅ Silver Layer Transformation Complete!")
print("=" * 80)
print("\n📌 Next Step: Run 03_Gold_Layer_Star_Schema notebook")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 31, Finished, Available, Finished)


✅ Silver Layer Transformation Complete!

📌 Next Step: Run 03_Gold_Layer_Star_Schema notebook


In [30]:
# Cleanup
df_taxi_clean.unpersist()
print("✅ Cleanup completed")

StatementMeta(, 50e044a6-361e-4fa8-86f8-76b5f5ff2cfc, 32, Finished, Available, Finished)

✅ Cleanup completed
